In [1]:
!rm -r smart-cities-drl
!git clone https://github.com/eescriba/smart-cities-drl
!cd smart-cities-drl/ && pip install -e .


Cloning into 'smart-cities-drl'...
remote: Enumerating objects: 573, done.
remote: Counting objects: 100% (573/573), done.
remote: Compressing objects: 100% (401/401), done.
remote: Total 573 (delta 200), reused 501 (delta 137), pack-reused 0
Receiving objects: 100% (573/573), 860.17 KiB | 28.67 MiB/s, done.
Resolving deltas: 100% (200/200), done.
Obtaining file:///content/smart-cities-drl
  Found existing installation: smart-cities-drl 0.1.0
    Can't uninstall 'smart-cities-drl'. No files were found to uninstall.
  Running setup.py develop for smart-cities-drl


In [ ]:
!pip install ray[rllib]

In [2]:
import argparse
from gym.spaces import Discrete, Tuple
import logging
import os

import random
import ray
from ray import tune
from ray.tune import function
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune import run, sample_from
from ray.tune.schedulers import PopulationBasedTraining


import sys
sys.path.insert(0,'./smart-cities-drl/src/')

from smartcab.henv import SmartCabEnv, HierarchicalSmartCabEnv


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# Postprocess the perturbed config to ensure it's still valid used if PBT.
def explore(config):
    # ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["sgd_minibatch_size"] * 2:
        config["train_batch_size"] = config["sgd_minibatch_size"] * 2
    # ensure we run at least one sgd iter
    if config["num_sgd_iter"] < 1:
        config["num_sgd_iter"] = 1
    return config

pbt = PopulationBasedTraining(
    time_attr="time_total_s",
    perturbation_interval=120,
    resample_probability=0.25,

    # Specifies the mutations of these hyperparams
    hyperparam_mutations={
        "lambda": lambda: random.uniform(0.9, 1.0),
        "clip_param": lambda: random.uniform(0.01, 0.5),
        "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
        "num_sgd_iter": lambda: random.randint(1, 30),
        "sgd_minibatch_size": lambda: random.randint(128, 16384),
        "train_batch_size": lambda: random.randint(2000, 160000),
    },
    custom_explore_fn=explore)


In [ ]:


stop = {
    "training_iteration": 200,
    "timesteps_total": 100000,
    "episode_reward_mean": 0.0,
}

env = SmartCabEnv(None)

def policy_mapping_fn(agent_id):
    if agent_id.startswith("low_level_"):
        return "low_level_policy"
    else:
        return "high_level_policy"

config = {
    "env": HierarchicalSmartCabEnv,
    "horizon": 99, 
    "num_workers": 0,
    "entropy_coeff": 0.01,
    "kl_coeff": 1.0,
    "model": {
        "free_log_std": True
    },
    "num_sgd_iter": 10,
    "sgd_minibatch_size": 128,
    "lambda": sample_from(lambda spec: random.uniform(0.9, 1.0)),
    "clip_param": sample_from(lambda spec: random.uniform(0.1, 0.5)),
    "lr": sample_from(lambda spec: random.uniform(1e-3, 1e-5)),
    "train_batch_size": sample_from(
        lambda spec: random.randint(1000, 60000)),
    "multiagent": {
        "policies": {
            "high_level_policy": (None, env.observation_space,
                                   env.action_space, {
                                      "gamma": 0.9
                                  }),
            "low_level_policy": (None,
                                  Tuple([
                                      env.observation_space,
                                       env.action_space,
                                  ]), env.action_space, {
                                      "gamma": 0.0
                                  }),
        },
        "policy_mapping_fn": function(policy_mapping_fn),
    },
    "framework": "tf" ,
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": 1,
}

results = tune.run("PPO",num_samples=8, scheduler=pbt, metric="episode_reward_mean", mode="max", stop=stop, config=config)

Actions:  6
Targets:  4
States:  500


2021-04-09 17:50:54,460	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
/usr/local/lib/python3.7/dist-packages/ray/tune/utils/util.py:634: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if np.isnan(value):


Trial name,status,loc,clip_param,lambda,lr,train_batch_size
PPO_HierarchicalSmartCabEnv_2348f_00000,RUNNING,,0.237024,0.94129,6.91066e-05,40969


(pid=5917) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=5917) Instructions for updating:
(pid=5917) non-resource variables are not supported in the long term
(pid=5917) 2021-04-09 17:51:00,163	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=5917) 2021-04-09 17:51:00,164	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=5917) Actions:  6
(pid=5917) Targets:  4
(pid=5917) States:  500


...

Result for PPO_HierarchicalSmartCabEnv_2348f_00000:
  custom_metrics: {}
  date: 2021-04-09_17-54-16
  done: false
  episode_len_mean: 99.0
  episode_reward_max: -285.0
  episode_reward_mean: -496.52415458937196
  episode_reward_min: -760.0
  episodes_this_iter: 414
  episodes_total: 828
  experiment_id: 78c7af00b9c94a478ab8a214bb4b9c25
  hostname: 056f531b774a
  info:
    learner:
      high_level_policy:
        cur_kl_coeff: 0.5
        cur_lr: 6.910657248226926e-05
        entropy: 1.78268563747406
        entropy_coeff: 0.009999999776482582
        kl: 0.003906698431819677
        model: {}
        policy_loss: -0.011853426694869995
        total_loss: -0.02772693522274494
        vf_explained_var: 0.2619088888168335
        vf_loss: 8.628056491488678e-09
      low_level_policy:
        cur_kl_coeff: 1.5
        cur_lr: 6.910657248226926e-05
        entropy: 1.7337336540222168
        entropy_coeff: 0.009999999776482582
        kl: 0.011258479207754135
        model: {}
        po

Trial name,status,loc,clip_param,lambda,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_HierarchicalSmartCabEnv_2348f_00000,PAUSED,,0.237024,0.94129,6.91066e-05,40969,2,188.932,82000,-496.524,-285,-760,99
PPO_HierarchicalSmartCabEnv_2348f_00001,PENDING,,0.370803,0.91771,0.000358753,49912,,,,,,,


(pid=5918) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=5918) Instructions for updating:
(pid=5918) non-resource variables are not supported in the long term
(pid=5918) 2021-04-09 17:54:20,532	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=5918) 2021-04-09 17:54:20,532	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=5918) Actions:  6
(pid=5918) Targets:  4
(pid=5918) States:  500


(pid=5918) 2021-04-09 17:54:21,192	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=5918) WARNING:tensorflow:
(pid=5918) The following Variables were used a Lambda layer's call (lambda), but
(pid=5918) are not present in its tracked objects:
(pid=5918)   <tf.Variable 'high_level_policy/log_std:0' shape=(3,) dtype=float32>
(pid=5918) It is possible that this is intended behavior, but it is more likely
(pid=5918) an omission. This is a strong indication that this layer should be
(pid=5918) formulated as a subclassed Layer rather than a Lambda layer.
(pid=5918) 2021-04-09 17:54:22,691	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=5918) WARNING:tensorflow:
(pid=5918) The following Variables were used a Lambda layer's call (lambda_1), but
(pid=5918) are not pr

In [ ]:
!rllib train --env=HierarchicalSmartCabEnv --run=PPO --config config

In [6]:
import ray.rllib.agents.ppo as ppo
import json
import shutil

CHECKPOINT_ROOT = "tmp/ppo/taxi"
shutil.rmtree(CHECKPOINT_ROOT, ignore_errors=True, onerror=None)

config={
      "num_gpus": 1,
      "env": SmartCabEnv,
      "horizon": 99
}
agent = ppo.PPOTrainer(config)
N_ITER = 50
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(CHECKPOINT_ROOT)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

(pid=4835) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=4835) Instructions for updating:
(pid=4835) non-resource variables are not supported in the long term
(pid=4834) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=4834) Instructions for updating:
(pid=4834) non-resource variables are not supported in the long term
(pid=4835) 2021-04-09 17:36:31,445	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=4834) 2021-04-09 17:36:31,536	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been d

(pid=4835) Actions:  6
(pid=4835) Targets:  4
(pid=4835) States:  500
(pid=4834) Actions:  6
(pid=4834) Targets:  4
(pid=4834) States:  500


2021-04-09 17:36:33,573	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-04-09 17:36:36,798	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=4835) 2021-04-09 17:36:36,807	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


(pid=4834) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/ray/rllib/policy/tf_policy.py:928: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=4834) Instructions for updating:
(pid=4834) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=4835) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/ray/rllib/policy/tf_policy.py:928: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=4835) Instructions for updating:
(pid=4835) Prefer Variable.assign which has equivalent behavior in 2.X.


  1: Min/Mean/Max reward: -954.0000/-682.7750/-517.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_1/checkpoint-1
  2: Min/Mean/Max reward: -954.0000/-610.3375/-308.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_2/checkpoint-2
  3: Min/Mean/Max reward: -954.0000/-513.9600/-308.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_3/checkpoint-3
  4: Min/Mean/Max reward: -745.0000/-396.9200/-194.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_4/checkpoint-4
  5: Min/Mean/Max reward: -555.0000/-311.0400/-156.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_5/checkpoint-5
  6: Min/Mean/Max reward: -422.0000/-246.4400/-118.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_6/checkpoint-6
  7: Min/Mean/Max reward: -327.0000/-200.8400/-99.0000, len mean:  99.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_7/checkpoint-7
  8: Min/Mean/Max reward: -308.0000/-172.9100/-99

KeyboardInterrupt: ignored

In [ ]:
"policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

In [5]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2021-04-09 17:36:23,015	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 51633,
 'node_id': '3f386c993d2c5fca12f65842f1dc69b55fb87f5b98ac62669f84e623',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-04-09_17-36-22_259695_4666/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-04-09_17-36-22_259695_4666/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-04-09_17-36-22_259695_4666',
 'webui_url': '127.0.0.1:8265'}